# CARLA Autonomous Vehicle with LiDAR-Camera Fusion

**AI-controlled vehicle using LiDAR-to-camera projection for obstacle detection and navigation**

Output: `output/YYYY-MM-DD_HH-MM-SS/lidar_projection/camera_{front,left,right}/`

## 1. Imports & Server Connection

In [1]:
import time
import carla
import numpy as np

from src import (
    Config, connect_to_carla, setup_synchronous_mode, restore_async_mode,
    clear_environment, reload_environment, spawn_vehicle, spawn_cameras,
    spawn_lidar, create_sensor_queues, project_lidar_to_camera,
    save_projection_image, update_spectator_view,
    lidar_density_to_waypoint,
)

from src.sensors import create_camera_callback, create_lidar_callback
from src.projection import image_to_array
from agents.navigation.controller import VehiclePIDController

config = Config()

# Connect to CARLA and enable synchronous mode
client, world = connect_to_carla(config)
original_settings, traffic_manager = setup_synchronous_mode(world, client, config)

print(f"✓ Connected to CARLA | Map: {world.get_map().name} | {config.fps} FPS sync mode")

✓ Connected to CARLA | Map: Carla/Maps/Town10HD_Opt | 20 FPS sync mode


---

## 2. Main Pipeline

Setup environment, spawn vehicle & sensors, run simulation with LiDAR-based steering.

### 1. Setup

In [6]:
# Environment setup (optional: clear/reload)
clear_environment(world)  # Remove buildings, props, etc.
# reload_environment(world)  # Uncomment to restore

# Spawn vehicle and sensors
vehicle = spawn_vehicle(world, config)
cameras, K = spawn_cameras(world, vehicle, config)
lidar = spawn_lidar(world, vehicle, config)
update_spectator_view(world, vehicle)
world.tick()

# Setup sensor queues and output directory
camera_queues, lidar_queue = create_sensor_queues(cameras)
output_dir = config.get_output_dir()
output_dirs = config.create_output_dirs(output_dir)

for cam_name, cam in cameras.items():
    cam.listen(create_camera_callback(camera_queues, cam_name))
lidar.listen(create_lidar_callback(lidar_queue))

print(f"✓ Vehicle + {len(cameras)} cameras + LiDAR spawned")
print(f"✓ Output: {output_dir}/")

✓ Vehicle + 3 cameras + LiDAR spawned
✓ Output: output/2026-01-31_12-05-53/


### 2. Simulation

In [ ]:
# Target speed settings
target_speed_kmh = config.throttle * 100

# Run simulation with LiDAR-based direct steering (no PID/waypoint snapping)
frame_counter, tick_count = 0, 0
current_projection_counts = {}

print(f"✓ Running {config.simulation_duration}s | Target speed: {target_speed_kmh} km/h | Controller: Direct LiDAR Steering\n")

try:
    while tick_count < config.total_ticks:
        world.tick()
        tick_count += 1
        
        # Wait for sensor data
        start = time.time()
        while time.time() - start < 2.0:
            if all(camera_queues[c].qsize() > 0 for c in cameras) and lidar_queue.qsize() > 0:
                break
            time.sleep(0.001)
        
        should_save = (tick_count % config.capture_interval_ticks == 0)
        
        # Collect and clear queues
        camera_images = {n: camera_queues[n].get() for n in cameras if not camera_queues[n].empty()}
        for q in camera_queues.values():
            while not q.empty(): q.get()
        lidar_data = lidar_queue.get() if not lidar_queue.empty() else None
        while not lidar_queue.empty(): lidar_queue.get()
        
        # Process sensors
        if len(camera_images) == len(cameras) and lidar_data:
            projection_counts = {}
            for cam_name in cameras:
                proj_array, _, weighted_density = project_lidar_to_camera(
                    lidar_data, lidar, cameras[cam_name], image_to_array(camera_images[cam_name]), K, config)
                projection_counts[cam_name] = weighted_density
                if should_save:
                    save_projection_image(cam_name, proj_array, frame_counter, output_dirs)
            
            if should_save:
                frame_counter += 1
            current_projection_counts = projection_counts
        
        # Direct LiDAR-based steering (bypasses waypoint snapping)
        if current_projection_counts:
            # Step 1: Convert LiDAR density to target waypoint (local coordinates)
            waypoint_local, direction, analysis = lidar_density_to_waypoint(
                current_projection_counts, vehicle, config
            )
            
            # Step 2: Convert local waypoint to world coordinates
            vehicle_transform = vehicle.get_transform()
            forward_vec = vehicle_transform.get_forward_vector()
            right_vec = vehicle_transform.get_right_vector()
            vehicle_loc = vehicle_transform.location
            
            world_x = vehicle_loc.x + waypoint_local[0] * forward_vec.x + waypoint_local[1] * right_vec.x
            world_y = vehicle_loc.y + waypoint_local[0] * forward_vec.y + waypoint_local[1] * right_vec.y
            world_z = vehicle_loc.z
            
            # Step 3: Calculate steering directly from target (NO waypoint snapping)
            target_vec = np.array([world_x - vehicle_loc.x, world_y - vehicle_loc.y])
            forward_np = np.array([forward_vec.x, forward_vec.y])
            
            # Cross product for steering direction, dot product for forward alignment
            cross = forward_np[0] * target_vec[1] - forward_np[1] * target_vec[0]
            dot = np.dot(forward_np, target_vec)
            angle = np.arctan2(cross, dot)
            
            # Step 4: Create control signal directly
            control_signal = carla.VehicleControl()
            control_signal.steer = float(np.clip(angle / np.radians(70), -1.0, 1.0))
            control_signal.throttle = config.throttle
            control_signal.brake = 0.0
            
            vehicle.apply_control(control_signal)
            
            # Log with direction decision from LiDAR
            print(f"[Tick {tick_count}] {direction} | Steer: {control_signal.steer:+.3f} | Target: ({world_x:.2f}, {world_y:.2f})")
        
        update_spectator_view(world, vehicle)

except KeyboardInterrupt:
    print("\n✗ Interrupted")

print(f"\n✓ Done! {frame_counter} frames saved")

✓ Running 30.0s | Target speed: 30.0 km/h | Controller: Direct LiDAR Steering

[Tick 1] RIGHT | Steer: +0.060 | Target: (-54.60, 25.27)
[Tick 2] STRAIGHT | Steer: +0.000 | Target: (-54.60, 24.53)
[Tick 3] LEFT | Steer: -0.077 | Target: (-54.60, 23.59)
[Tick 4] RIGHT | Steer: +0.137 | Target: (-54.61, 26.23)
[Tick 5] STRAIGHT | Steer: +0.000 | Target: (-54.60, 24.53)
[Tick 6] RIGHT | Steer: +0.001 | Target: (-54.60, 24.55)
[Tick 7] STRAIGHT | Steer: +0.000 | Target: (-54.60, 24.53)
[Tick 8] STRAIGHT | Steer: +0.000 | Target: (-54.60, 24.53)
[Tick 9] STRAIGHT | Steer: +0.000 | Target: (-54.60, 24.53)
[Tick 10] STRAIGHT | Steer: +0.000 | Target: (-54.60, 24.53)
[Tick 11] STRAIGHT | Steer: +0.000 | Target: (-54.60, 24.53)
[Tick 12] STRAIGHT | Steer: +0.000 | Target: (-54.56, 24.53)
[Tick 13] STRAIGHT | Steer: +0.000 | Target: (-54.40, 24.54)
[Tick 14] STRAIGHT | Steer: +0.000 | Target: (-54.22, 24.54)
[Tick 15] STRAIGHT | Steer: +0.000 | Target: (-54.02, 24.54)
[Tick 16] STRAIGHT | Steer: 

NameError: name 'logger' is not defined

---

## 3. Cleanup & Shutdown

Stop sensors, destroy actors, restore async mode.

In [8]:
restore_async_mode(world, original_settings, traffic_manager)

for cam in cameras.values():
    cam.stop()
lidar.stop()
time.sleep(0.5)

for cam in cameras.values():
    cam.destroy()
lidar.destroy()
vehicle.destroy()
world.tick()

print(f"✓ Cleanup done | Output: {output_dir}/lidar_projection/")

✓ Cleanup done | Output: output/2026-01-31_12-05-53/lidar_projection/
